# Day8_1: 시계열 분석 기법 (resample, rolling, 분해)

## 📚 학습 목표

**Part 1: 기초**
1. resample()로 시계열 데이터 리샘플링하기
2. 업샘플링과 다운샘플링의 차이 이해하기
3. 다양한 집계 함수로 주기 변환하기
4. rolling()으로 이동 평균 계산하기
5. expanding()으로 누적 통계 구하기

**Part 2: 심화**
1. 지수 가중 이동 평균(EWM) 활용하기
2. 시계열 분해로 추세와 계절성 추출하기
3. Plotly로 시계열 분해 결과 시각화하기
4. 이동 통계를 활용한 이상치 탐지하기

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| resample | 시간 단위 통합/확장 | 일별 매출 → 월별 매출, 시간별 → 분별 |
| rolling | 노이즈 제거, 추세 파악 | 주가 이동평균선, 7일 평균 매출 |
| expanding | 누적 성과 추적 | 연초부터 현재까지 누적 매출 |
| 시계열 분해 | 패턴 분석, 예측 모델링 | 계절성 제거 후 추세 예측 |

**분석가 관점**: 주식 차트의 이동평균선, 쇼핑몰 계절성 분석, 웹 트래픽 추세 파악 등 시계열 패턴을 발견하고 노이즈를 제거하는 필수 기법입니다!

---

In [24]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta

# 한글 폰트 설정 (Plotly)
import plotly.io as pio
pio.templates.default = "plotly_white"

print("✅ 라이브러리 로드 완료")

✅ 라이브러리 로드 완료


# Part 1: 기초

---

## 1.1 resample()로 리샘플링하기

### resample()이란?

- **시계열 데이터의 주기를 변경**하는 메서드
- **다운샘플링**: 고빈도 → 저빈도 (예: 일별 → 월별)
- **업샘플링**: 저빈도 → 고빈도 (예: 월별 → 일별)
- DatetimeIndex가 필요함

In [25]:
# 일별 매출 데이터 생성
np.random.seed(42)    
dates = pd.date_range(start='2024-01-01', end='2024-01-31', freq='D')
daily_sales = pd.DataFrame({
    'date': dates,
    'sales': np.random.randint(100, 300, size=len(dates))
})

# DatetimeIndex로 설정
daily_sales.set_index('date', inplace=True)
print("일별 매출 데이터 (처음 5개):")
print(daily_sales.head())
print(f"\n총 {len(daily_sales)}개 레코드")

일별 매출 데이터 (처음 5개):
            sales
date             
2024-01-01    202
2024-01-02    279
2024-01-03    192
2024-01-04    114
2024-01-05    206

총 31개 레코드


### 다운샘플링: 일별 → 주별

In [26]:
# 주별(Weekly) 집계 - 합계
weekly_sales_sum = daily_sales.resample('W').sum()      #resample은 날짜기반 집계라고 볼 수 있음
print("주별 매출 합계:")                                 #resample하려면 날짜들이 datetime 인덱스로 변환된 상태여야함. 
print(weekly_sales_sum)

# 주별 평균
weekly_sales_mean = daily_sales.resample('W').mean()
print("\n주별 매출 평균:")
print(weekly_sales_mean)

주별 매출 합계:
            sales
date             
2024-01-07   1452
2024-01-14   1319
2024-01-21   1373
2024-01-28   1581
2024-02-04    466

주별 매출 평균:
                 sales
date                  
2024-01-07  207.428571
2024-01-14  188.428571
2024-01-21  196.142857
2024-01-28  225.857143
2024-02-04  155.333333


### 다양한 주기 코드

| 코드 | 의미 | 예시 |
|------|------|------|
| `'D'` | 일(Day) | 매일 |
| `'W'` | 주(Week) | 매주 일요일 |
| `'M'` | 월말(Month End) | 각 월 마지막 날 |
| `'MS'` | 월초(Month Start) | 각 월 첫날 |
| `'Q'` | 분기(Quarter) | 3개월 |
| `'Y'` | 연말(Year End) | 12월 31일 |
| `'H'` | 시간(Hour) | 매시간 |
| `'T'` or `'min'` | 분(Minute) | 매분 |

In [27]:
# 월별 집계
monthly_sales = daily_sales.resample('ME').agg({
    'sales': ['sum', 'mean', 'max', 'min', 'count']
})

print("월별 매출 집계:")
print(monthly_sales)

월별 매출 집계:
           sales                            
             sum        mean  max  min count
date                                        
2024-01-31  6191  199.709677  291  101    31


### 💡 실무 예시: 일별 주가 → 월별 요약

주식 데이터를 월별로 집계하여 OHLC (Open, High, Low, Close) 차트 생성

In [28]:
# 주가 데이터 시뮬레이션 (종가)
np.random.seed(100)
dates_stock = pd.date_range('2024-01-01', '2024-12-31', freq='D')
stock_price = pd.DataFrame({
    'close': 50000 + np.cumsum(np.random.randn(len(dates_stock)) * 500)     #cumsum 누적합계
}, index=dates_stock)                                                       np.cumsum([1,2,3,4,5]) 하면 [1,3,6,10,15]가 됨

SyntaxError: invalid syntax (1520823956.py, line 6)

In [30]:
# 월별 OHLC 계산
monthly_ohlc = stock_price.resample('M').agg({
    'close': ['first', 'max', 'min', 'last']
})

monthly_ohlc.columns = ['Open', 'High', 'Low', 'Close']
print("월별 OHLC:")
print(monthly_ohlc.head())

NameError: name 'stock_price' is not defined

---

## 1.2 업샘플링과 결측치 처리

### 업샘플링이란?

- 낮은 빈도 → 높은 빈도로 변환
- 새로 생긴 인덱스는 **결측치(NaN)**가 됨
- `.ffill()`, `.bfill()`, `.interpolate()`로 채우기

In [32]:
# 월별 데이터 생성
monthly_data = pd.DataFrame({
    'revenue': [1000, 1200, 1100, 1300]
}, index=pd.date_range('2024-01', periods=4, freq='MS'))

print("월별 데이터:")
print(monthly_data)

월별 데이터:
            revenue
2024-01-01     1000
2024-02-01     1200
2024-03-01     1100
2024-04-01     1300


In [31]:
# 일별로 업샘플링 (결측치 발생)
daily_upsampled = monthly_data.resample('D').asfreq()       #데이터가 monthly로 되어있는데 이걸 daily로 바꾸려는 것(적은데이터 -> 많은데이터)
print("일별 업샘플링 (처음 10개):")                                 #이런걸 upsampling이라고 함
print(daily_upsampled.head(10))

# 결측치 처리
monthly_data = monthly_data.fillna(method='ffill')          #ffill - 앞의 숫자로 채워지는 거임
print("결측치 처리 후 (처음 10개):")
print(monthly_data.head(10))

NameError: name 'monthly_data' is not defined

In [33]:
# Forward Fill (앞 값으로 채우기)
daily_ffill = monthly_data.resample('D').ffill()
print("Forward Fill (처음 35일):")
print(daily_ffill.head(35))

Forward Fill (처음 35일):
            revenue
2024-01-01     1000
2024-01-02     1000
2024-01-03     1000
2024-01-04     1000
2024-01-05     1000
2024-01-06     1000
2024-01-07     1000
2024-01-08     1000
2024-01-09     1000
2024-01-10     1000
2024-01-11     1000
2024-01-12     1000
2024-01-13     1000
2024-01-14     1000
2024-01-15     1000
2024-01-16     1000
2024-01-17     1000
2024-01-18     1000
2024-01-19     1000
2024-01-20     1000
2024-01-21     1000
2024-01-22     1000
2024-01-23     1000
2024-01-24     1000
2024-01-25     1000
2024-01-26     1000
2024-01-27     1000
2024-01-28     1000
2024-01-29     1000
2024-01-30     1000
2024-01-31     1000
2024-02-01     1200
2024-02-02     1200
2024-02-03     1200
2024-02-04     1200


In [34]:
# Interpolate (선형 보간)
daily_interpolated = monthly_data.resample('D').interpolate(method='linear')        #선형으로 채워주는 거 
print("선형 보간 (처음 35일):")
print(daily_interpolated.head(35))

선형 보간 (처음 35일):
                revenue
2024-01-01  1000.000000
2024-01-02  1006.451613
2024-01-03  1012.903226
2024-01-04  1019.354839
2024-01-05  1025.806452
2024-01-06  1032.258065
2024-01-07  1038.709677
2024-01-08  1045.161290
2024-01-09  1051.612903
2024-01-10  1058.064516
2024-01-11  1064.516129
2024-01-12  1070.967742
2024-01-13  1077.419355
2024-01-14  1083.870968
2024-01-15  1090.322581
2024-01-16  1096.774194
2024-01-17  1103.225806
2024-01-18  1109.677419
2024-01-19  1116.129032
2024-01-20  1122.580645
2024-01-21  1129.032258
2024-01-22  1135.483871
2024-01-23  1141.935484
2024-01-24  1148.387097
2024-01-25  1154.838710
2024-01-26  1161.290323
2024-01-27  1167.741935
2024-01-28  1174.193548
2024-01-29  1180.645161
2024-01-30  1187.096774
2024-01-31  1193.548387
2024-02-01  1200.000000
2024-02-02  1196.551724
2024-02-03  1193.103448
2024-02-04  1189.655172


### 💡 실무 예시: 주별 예산을 일별로 분배

마케팅 예산을 주별로 받았는데, 일별로 균등 분배하고 싶을 때

In [35]:
# 주별 예산
weekly_budget = pd.DataFrame({
    'budget': [7000, 10500, 14000, 10500]  # 단위: 만원
}, index=pd.date_range('2024-01-07', periods=4, freq='W'))

print("주별 예산:")
print(weekly_budget)

# 일별로 보간 후 7로 나누기 (주당 7일)
daily_budget = weekly_budget.resample('D').interpolate() / 7
print("\n일별 예산 (처음 14일):")
print(daily_budget.head(14))

주별 예산:
            budget
2024-01-07    7000
2024-01-14   10500
2024-01-21   14000
2024-01-28   10500

일별 예산 (처음 14일):
                 budget
2024-01-07  1000.000000
2024-01-08  1071.428571
2024-01-09  1142.857143
2024-01-10  1214.285714
2024-01-11  1285.714286
2024-01-12  1357.142857
2024-01-13  1428.571429
2024-01-14  1500.000000
2024-01-15  1571.428571
2024-01-16  1642.857143
2024-01-17  1714.285714
2024-01-18  1785.714286
2024-01-19  1857.142857
2024-01-20  1928.571429


---

## 1.3 rolling()으로 이동 평균

### 이동 평균(Moving Average)이란?

- **일정 기간의 평균**을 계속 계산하며 이동
- 노이즈 제거, 추세 파악에 유용
- 주식 차트의 5일선, 20일선, 60일선 등

In [ ]:
# 일별 매출 데이터 (노이즈가 있는)
np.random.seed(50)
dates_ma = pd.date_range('2024-01-01', periods=60, freq='D')
sales_noisy = pd.DataFrame({
    'sales': 200 + np.cumsum(np.random.randn(60) * 10) + np.random.randn(60) * 20
}, index=dates_ma)

print("원본 매출 (처음 10개):")
print(sales_noisy.head(10))

원본 매출 (처음 10개):
                 sales
2024-01-01  186.138773
2024-01-02  184.772070
2024-01-03  192.808797
2024-01-04  144.358862
2024-01-05  172.435542
2024-01-06  194.746173
2024-01-07  165.559219
2024-01-08  171.216405
2024-01-09  144.839721
2024-01-10  143.971097


In [ ]:
# 7일 이동 평균
sales_noisy['MA_7'] = sales_noisy['sales'].rolling(window=7).mean()

# 30일 이동 평균
sales_noisy['MA_30'] = sales_noisy['sales'].rolling(window=30).mean()

print("이동 평균 추가 (처음 35개):")
print(sales_noisy.head(35))

이동 평균 추가 (처음 35개):
                 sales        MA_7       MA_30
2024-01-01  186.138773         NaN         NaN
2024-01-02  184.772070         NaN         NaN
2024-01-03  192.808797         NaN         NaN
2024-01-04  144.358862         NaN         NaN
2024-01-05  172.435542         NaN         NaN
2024-01-06  194.746173         NaN         NaN
2024-01-07  165.559219  177.259919         NaN
2024-01-08  171.216405  175.128153         NaN
2024-01-09  144.839721  169.423531         NaN
2024-01-10  143.971097  162.446717         NaN
2024-01-11  155.092275  163.980062         NaN
2024-01-12  164.305609  162.818643         NaN
2024-01-13  180.395229  160.768508         NaN
2024-01-14  172.770852  161.798741         NaN
2024-01-15  182.335828  163.387230         NaN
2024-01-16  157.252488  165.160482         NaN
2024-01-17  243.362021  179.359186         NaN
2024-01-18  233.680013  190.586006         NaN
2024-01-19  229.265779  199.866030         NaN
2024-01-20  188.832155  201.071305       

### rolling() 파라미터

- `window`: 윈도우 크기 (필수)
- `min_periods`: 최소 데이터 개수 (기본값: window와 동일)
- `center`: True면 중앙 정렬 (기본값: False)
- `.mean()`, `.sum()`, `.std()`, `.max()`, `.min()` 등 집계 함수

In [ ]:
# 이동 표준편차 (변동성 측정)
sales_noisy['rolling_std'] = sales_noisy['sales'].rolling(window=7).std()

print("이동 표준편차 (처음 15개):")
print(sales_noisy[['sales', 'rolling_std']].head(15))

### 💡 실무 예시: 주가 골든크로스 탐지

단기 이동평균선이 장기 이동평균선을 상향 돌파하면 매수 신호

In [ ]:
# 주가 시뮬레이션
np.random.seed(123)
stock_data = pd.DataFrame({
    'price': 50000 + np.cumsum(np.random.randn(100) * 500)
}, index=pd.date_range('2024-01-01', periods=100, freq='D'))

# 5일선, 20일선
stock_data['MA5'] = stock_data['price'].rolling(5).mean()
stock_data['MA20'] = stock_data['price'].rolling(20).mean()

# 골든크로스 탐지 (5일선 > 20일선 && 전날은 반대)
stock_data['golden_cross'] = (
    (stock_data['MA5'] > stock_data['MA20']) &
    (stock_data['MA5'].shift(1) <= stock_data['MA20'].shift(1))
)

print("골든크로스 발생일:")
print(stock_data[stock_data['golden_cross']])

---

## 1.4 expanding()으로 누적 통계

### expanding()이란?

- **처음부터 현재까지** 모든 데이터를 대상으로 계산
- 누적 평균, 누적 합계, 누적 최대값 등
- `window`가 계속 확장됨
- window rolling 함수라고 볼 수 있음

In [ ]:
# 일별 매출
daily_revenue = pd.DataFrame({
    'revenue': [100, 120, 90, 150, 130, 140, 110, 160, 180, 170]
}, index=pd.date_range('2024-01-01', periods=10, freq='D'))

# 누적 합계 (연초부터 현재까지)
daily_revenue['cumulative_sum'] = daily_revenue['revenue'].expanding().sum()

# 누적 평균
daily_revenue['cumulative_mean'] = daily_revenue['revenue'].expanding().mean()

print("누적 통계:")
print(daily_revenue)

### 💡 실무 예시: 연초부터 누적 매출 목표 달성률

In [ ]:
# 일별 목표 매출
target_daily = 150  # 만원

# 누적 매출 vs 누적 목표
daily_revenue['cumulative_target'] = target_daily * (daily_revenue.index - daily_revenue.index[0]).days + target_daily
daily_revenue['achievement_rate'] = (daily_revenue['cumulative_sum'] / daily_revenue['cumulative_target'] * 100).round(2)

print("누적 매출 목표 달성률:")
print(daily_revenue[['revenue', 'cumulative_sum', 'cumulative_target', 'achievement_rate']])

---

# Part 2: 심화

---

## 2.1 지수 가중 이동 평균 (EWM)

### EWM (Exponentially Weighted Moving Average)

- **최근 데이터에 더 큰 가중치**를 부여
- 단순 이동평균보다 빠르게 최근 변화를 반영
- 금융 시계열 분석에서 널리 사용
- `span` 파라미터로 가중치 조절 (클수록 평활화 효과 증가) => hyer parameter라고 함(사람이 알 수 없음, 실험을 통해서 알 수 있음)

In [36]:
# 주가 데이터
np.random.seed(200)
stock = pd.DataFrame({
    'price': 100 + np.cumsum(np.random.randn(50) * 2)
}, index=pd.date_range('2024-01-01', periods=50, freq='D'))

# 단순 이동평균 (SMA)
stock['SMA_10'] = stock['price'].rolling(10).mean()

# 지수 가중 이동평균 (EWM)
stock['EWM_10'] = stock['price'].ewm(span=10, adjust=False).mean()

print("SMA vs EWM (처음 15개):")
print(stock.head(15))

stock.plot.line()

SMA vs EWM (처음 15개):
                 price      SMA_10      EWM_10
2024-01-01   97.098104         NaN   97.098104
2024-01-02  100.920010         NaN   97.792996
2024-01-03  102.343768         NaN   98.620409
2024-01-04  101.848292         NaN   99.207297
2024-01-05  102.571224         NaN   99.818920
2024-01-06  102.505325         NaN  100.307357
2024-01-07  102.062631         NaN  100.626498
2024-01-08  103.017145         NaN  101.061161
2024-01-09  101.633266         NaN  101.165180
2024-01-10  103.217278  101.721704  101.538289
2024-01-11  103.363776  102.348271  101.870195
2024-01-12  105.970348  102.853305  102.615678
2024-01-13  106.397311  103.258660  103.303247
2024-01-14  108.432009  103.917031  104.235750
2024-01-15  112.255433  104.885452  105.693874


ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

In [ ]:
# 시각화: SMA vs EWM 비교
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=stock.index, y=stock['price'],
    mode='lines', name='원본 가격',
    line=dict(color='lightgray', width=1)
))

fig.add_trace(go.Scatter(
    x=stock.index, y=stock['SMA_10'],
    mode='lines', name='SMA (10일)',
    line=dict(color='blue', width=2)
))

fig.add_trace(go.Scatter(
    x=stock.index, y=stock['EWM_10'],
    mode='lines', name='EWM (span=10)',
    line=dict(color='red', width=2, dash='dash')
))

fig.update_layout(
    title='단순 이동평균 vs 지수 가중 이동평균',
    xaxis_title='날짜',
    yaxis_title='가격',
    hovermode='x unified'
)

fig.show()

### 💡 실무 예시: 변동성(Volatility) 측정

주가의 표준편차를 EWM으로 계산하여 최근 변동성 추적

In [46]:
# 일별 수익률
stock['daily_return'] = stock['price'].pct_change() * 100

# 변동성 (EWM 표준편차)
stock['volatility'] = stock['daily_return'].ewm(span=20, adjust=False).std()

print("변동성 (처음 25개):")
print(stock[['price', 'daily_return', 'volatility']].head(25))

변동성 (처음 25개):
                 price  daily_return  volatility
2024-01-01   97.098104           NaN         NaN
2024-01-02  100.920010      3.936129         NaN
2024-01-03  102.343768      1.410779    1.785692
2024-01-04  101.848292     -0.484130    2.527774
2024-01-05  102.571224      0.709813    2.360056
2024-01-06  102.505325     -0.064247    2.393114
2024-01-07  102.062631     -0.431874    2.443406
2024-01-08  103.017145      0.935223    2.276951
2024-01-09  101.633266     -1.343348    2.440138
2024-01-10  103.217278      1.558556    2.259184
2024-01-11  103.363776      0.141932    2.181583
2024-01-12  105.970348      2.521746    2.054069
2024-01-13  106.397311      0.402908    1.979484
2024-01-14  108.432009      1.912358    1.863311
2024-01-15  112.255433      3.526102    1.846195
2024-01-16  111.196089     -0.943690    1.952308
2024-01-17  114.880360      3.313309    1.919399
2024-01-18  112.765890     -1.840585    2.130401
2024-01-19  111.040057     -1.530456    2.213200
2024-0

---

## 2.2 시계열 분해 (Decomposition)

### 시계열 분해란?

시계열 데이터를 3가지 요소로 분리:   ⭐T.S.R로 부르기도 함

1. **Trend (추세)**: 장기적인 상승/하락 패턴
2. **Seasonality (계절성)**: 주기적으로 반복되는 패턴
3. **Residual (잔차)**: 설명되지 않는 랜덤 노이즈

**분해 모델**:
- **Additive (가산)**: `Y = Trend + Seasonality + Residual`
- **Multiplicative (승산)**: `Y = Trend × Seasonality × Residual`

*적합하다 = FIT,  잔차가 크면 그 모델은 fit하지 않음

In [51]:
!uv add statsmodels     
#파이썬에 있는 통계함수 라이브러리, R을 본땄음, 그중에서도 tsa(time series analytics는 시계열 분석에 특화된 분석)

Resolved 100 packages in 1ms
Audited 98 packages in 4ms


In [52]:
# statsmodels 설치 필요: uv add statsmodels
from statsmodels.tsa.seasonal import seasonal_decompose

# 계절성이 있는 데이터 생성 (2년치 월별)
np.random.seed(42)
dates_decomp = pd.date_range('2022-01', periods=24, freq='MS')

# 추세 (상승) + 계절성 (연말 피크) + 노이즈
trend = np.linspace(100, 200, 24)
seasonality = 20 * np.sin(np.linspace(0, 4*np.pi, 24))
noise = np.random.randn(24) * 5

sales_seasonal = pd.DataFrame({
    'sales': trend + seasonality + noise
}, index=dates_decomp)

print("계절성 데이터:")
print(sales_seasonal)

계절성 데이터:
                 sales
2022-01-01  102.483571
2022-02-01  114.048184
2022-03-01  129.691799
2022-04-01  140.612003
2022-05-01  132.559935
2022-06-01  128.536467
2022-07-01  131.259688
2022-08-01  121.650197
2022-09-01  113.590018
2022-10-01  122.261553
2022-11-01  126.544453
2022-12-01  140.101503
2023-01-01  158.779660
2023-02-01  161.572057
2023-03-01  171.826658
2023-04-01  181.251172
2023-05-01  177.122821
2023-06-01  178.207613
2023-07-01  165.752727
2023-08-01  159.207779
2023-09-01  174.331390
2023-10-01  172.417762
2023-11-01  185.598136
2023-12-01  192.876259


In [55]:
# 시계열 분해 (Additive)
decomposition = seasonal_decompose(
    sales_seasonal['sales'],
    model='additive',
    period=12  # 12개월 주기 =>  이걸 넣어줘야 1년단위로 분석을 함
)

# 분해 결과
trend_component = decomposition.trend
seasonal_component = decomposition.seasonal
residual_component = decomposition.resid

print("추세 (Trend):")
print(trend_component.head(15))
print("\n계절성 (Seasonality):")
print(seasonal_component.head(15))
print("\n잔차 (Residual):")
print(residual_component.head(15))
pd.concat([sales_seasonal['sales'], trend_component, seasonal_component, residual_component], axis=1).plot.line()

추세 (Trend):
2022-01-01           NaN
2022-02-01           NaN
2022-03-01           NaN
2022-04-01           NaN
2022-05-01           NaN
2022-06-01           NaN
2022-07-01    127.623951
2022-08-01    131.949783
2022-09-01    135.685564
2022-10-01    139.134481
2022-11-01    142.684567
2022-12-01    146.610985
2023-01-01    150.117826
2023-02-01    153.119935
2023-03-01    157.215725
Freq: MS, Name: trend, dtype: float64

계절성 (Seasonality):
2022-01-01     8.598929
2022-02-01     8.389217
2022-03-01    14.548028
2022-04-01    19.351810
2022-05-01    10.673046
2022-06-01     7.098320
2022-07-01     3.572831
2022-08-01   -10.362491
2022-09-01   -22.158450
2022-10-01   -16.935833
2022-11-01   -16.203019
2022-12-01    -6.572387
2023-01-01     8.598929
2023-02-01     8.389217
2023-03-01    14.548028
Freq: MS, Name: seasonal, dtype: float64

잔차 (Residual):
2022-01-01         NaN
2022-02-01         NaN
2022-03-01         NaN
2022-04-01         NaN
2022-05-01         NaN
2022-06-01         NaN


ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

### 💡 실무 예시: 소매업 계절성 분석

연말 쇼핑 시즌, 명절 등의 계절 효과를 제거하고 순수한 성장 추세 파악

In [53]:
# 분해 결과를 DataFrame으로
decomp_df = pd.DataFrame({
    'original': sales_seasonal['sales'],
    'trend': trend_component,
    'seasonal': seasonal_component,
    'residual': residual_component
})

# 계절성 제거 데이터 (Deseasonalized)
decomp_df['deseasonalized'] = decomp_df['original'] - decomp_df['seasonal']

print("계절성 제거 데이터:")
print(decomp_df[['original', 'seasonal', 'deseasonalized']].head(15))

계절성 제거 데이터:
              original   seasonal  deseasonalized
2022-01-01  102.483571   8.598929       93.884642
2022-02-01  114.048184   8.389217      105.658967
2022-03-01  129.691799  14.548028      115.143771
2022-04-01  140.612003  19.351810      121.260193
2022-05-01  132.559935  10.673046      121.886889
2022-06-01  128.536467   7.098320      121.438147
2022-07-01  131.259688   3.572831      127.686856
2022-08-01  121.650197 -10.362491      132.012688
2022-09-01  113.590018 -22.158450      135.748469
2022-10-01  122.261553 -16.935833      139.197386
2022-11-01  126.544453 -16.203019      142.747472
2022-12-01  140.101503  -6.572387      146.673890
2023-01-01  158.779660   8.598929      150.180731
2023-02-01  161.572057   8.389217      153.182840
2023-03-01  171.826658  14.548028      157.278630


---

## 2.3 Plotly로 시계열 분해 시각화

### 4개 서브플롯으로 분해 결과 시각화

In [41]:
# 4개 서브플롯 생성
fig = make_subplots(
    rows=4, cols=1,
    subplot_titles=('원본 데이터', '추세 (Trend)', '계절성 (Seasonality)', '잔차 (Residual)'),
    vertical_spacing=0.08
)

# 1. 원본
fig.add_trace(
    go.Scatter(x=decomp_df.index, y=decomp_df['original'], name='원본', line=dict(color='black')),
    row=1, col=1
)

# 2. 추세
fig.add_trace(
    go.Scatter(x=decomp_df.index, y=decomp_df['trend'], name='추세', line=dict(color='blue')),
    row=2, col=1
)

# 3. 계절성
fig.add_trace(
    go.Scatter(x=decomp_df.index, y=decomp_df['seasonal'], name='계절성', line=dict(color='green')),
    row=3, col=1
)

# 4. 잔차
fig.add_trace(
    go.Scatter(x=decomp_df.index, y=decomp_df['residual'], name='잔차', mode='markers', marker=dict(color='red', size=4)),
    row=4, col=1
)

fig.update_layout(
    height=800,
    title_text="시계열 분해 (Additive Model)",
    showlegend=False
)

fig.show()

### 인터랙티브 비교: 원본 vs 계절성 제거

In [42]:
fig2 = go.Figure()

fig2.add_trace(go.Scatter(
    x=decomp_df.index, y=decomp_df['original'],
    mode='lines+markers', name='원본 데이터',
    line=dict(color='lightblue', width=2)
))

fig2.add_trace(go.Scatter(
    x=decomp_df.index, y=decomp_df['deseasonalized'],
    mode='lines+markers', name='계절성 제거',
    line=dict(color='darkblue', width=2, dash='dash')
))

fig2.add_trace(go.Scatter(
    x=decomp_df.index, y=decomp_df['trend'],
    mode='lines', name='추세선',
    line=dict(color='red', width=3)
))

fig2.update_layout(
    title='원본 vs 계절성 제거 vs 추세',
    xaxis_title='날짜',
    yaxis_title='매출 (만원)',
    hovermode='x unified'
)

fig2.show()

---

## 2.4 이동 통계로 이상치 탐지

### Bollinger Bands 기법

- 이동평균 ± (표준편차 × 2)
- 밴드를 벗어나면 이상치로 간주
- 주가 변동성, 센서 데이터 이상 탐지에 활용

In [56]:
# 센서 데이터 시뮬레이션 (온도)
np.random.seed(999)
sensor_data = pd.DataFrame({
    'temperature': 25 + np.random.randn(100) * 2
}, index=pd.date_range('2024-01-01', periods=100, freq='H'))

# 일부 이상치 주입
sensor_data.loc[sensor_data.index[20], 'temperature'] = 35  # 이상 고온
sensor_data.loc[sensor_data.index[50], 'temperature'] = 15  # 이상 저온
sensor_data.loc[sensor_data.index[80], 'temperature'] = 40  # 이상 고온



/tmp/ipykernel_49594/3308754175.py:5: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



In [ ]:

# Bollinger Bands 계산
window = 20
sensor_data['MA'] = sensor_data['temperature'].rolling(window).mean()
sensor_data['STD'] = sensor_data['temperature'].rolling(window).std()
sensor_data['Upper'] = sensor_data['MA'] + 2 * sensor_data['STD']
sensor_data['Lower'] = sensor_data['MA'] - 2 * sensor_data['STD']

# 이상치 플래그
sensor_data['anomaly'] = (
    (sensor_data['temperature'] > sensor_data['Upper']) |   # |라서 or조건임
    (sensor_data['temperature'] < sensor_data['Lower'])
)

print("이상치 탐지 결과:")
print(sensor_data[sensor_data['anomaly']])

In [44]:
# Bollinger Bands 시각화
fig3 = go.Figure()

# 온도 데이터
fig3.add_trace(go.Scatter(
    x=sensor_data.index, y=sensor_data['temperature'],
    mode='lines', name='온도',
    line=dict(color='gray', width=1)
))

# 이동평균
fig3.add_trace(go.Scatter(
    x=sensor_data.index, y=sensor_data['MA'],
    mode='lines', name='이동평균 (20)',
    line=dict(color='blue', width=2)
))

# Upper Band
fig3.add_trace(go.Scatter(
    x=sensor_data.index, y=sensor_data['Upper'],
    mode='lines', name='Upper Band',
    line=dict(color='red', width=1, dash='dash')
))

# Lower Band
fig3.add_trace(go.Scatter(
    x=sensor_data.index, y=sensor_data['Lower'],
    mode='lines', name='Lower Band',
    line=dict(color='red', width=1, dash='dash')
))

# 이상치 마커
anomalies = sensor_data[sensor_data['anomaly']]
fig3.add_trace(go.Scatter(
    x=anomalies.index, y=anomalies['temperature'],
    mode='markers', name='이상치',
    marker=dict(color='orange', size=12, symbol='x')
))

fig3.update_layout(
    title='Bollinger Bands를 활용한 이상치 탐지',
    xaxis_title='시간',
    yaxis_title='온도 (°C)',
    hovermode='x unified'
)

fig3.show()

### 💡 실무 예시: 웹 트래픽 이상 탐지

평소와 다른 급격한 트래픽 증가/감소를 탐지하여 DDoS 공격이나 시스템 장애 파악

In [45]:
# 웹 트래픽 시뮬레이션 (시간당 요청 수)
np.random.seed(555)
traffic = pd.DataFrame({
    'requests': np.random.poisson(lam=1000, size=200)
}, index=pd.date_range('2024-01-01', periods=200, freq='H'))

# DDoS 공격 시뮬레이션 (갑작스런 트래픽 증가)
traffic.loc[traffic.index[100:105], 'requests'] = np.random.poisson(lam=5000, size=5)

# 이동 평균 및 표준편차
traffic['MA'] = traffic['requests'].rolling(24).mean()
traffic['Upper'] = traffic['MA'] + 3 * traffic['requests'].rolling(24).std()

# 이상치 탐지
traffic['spike'] = traffic['requests'] > traffic['Upper']

print("트래픽 급증 탐지:")
print(traffic[traffic['spike']])

트래픽 급증 탐지:
                     requests           MA        Upper  spike
2024-01-05 04:00:00      5006  1170.083333  3622.284000   True
2024-01-05 05:00:00      5024  1338.000000  4736.275164   True


/tmp/ipykernel_49594/2566482806.py:5: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. 일별 매출을 주별로 집계하기 ⭐

**문제**: 다음 일별 매출 데이터를 주별로 다운샘플링하여 합계를 구하세요.

```python
dates = pd.date_range('2024-01-01', periods=14, freq='D')
sales = pd.DataFrame({'revenue': [100, 120, 90, 110, 130, 140, 95, 105, 125, 115, 135, 145, 100, 120]}, index=dates)
```

**기대 결과**: 주별 매출 합계 (2개 주)

In [ ]:
# 여기에 코드를 작성하세요


### Q2. 7일 이동 평균 계산하기 ⭐⭐

**문제**: 주어진 매출 데이터에 7일 이동 평균 컬럼을 추가하세요.

```python
dates = pd.date_range('2024-01-01', periods=30, freq='D')
sales = pd.DataFrame({'revenue': np.random.randint(80, 150, size=30)}, index=dates)
```

**기대 결과**: 'MA_7' 컬럼 추가

In [ ]:
# 여기에 코드를 작성하세요


### Q3. 누적 합계 구하기 ⭐⭐

**문제**: 일별 수익에 대해 연초부터 현재까지 누적 합계를 계산하세요.

```python
dates = pd.date_range('2024-01-01', periods=10, freq='D')
profit = pd.DataFrame({'profit': [50, -20, 30, 40, -10, 60, 20, 30, -15, 45]}, index=dates)
```

**기대 결과**: 'cumulative_profit' 컬럼 추가

In [ ]:
# 여기에 코드를 작성하세요


### Q4. 월별 OHLC 집계 ⭐⭐⭐

**문제**: 일별 주가 데이터를 월별로 다운샘플링하여 OHLC (Open, High, Low, Close)를 계산하세요.

```python
dates = pd.date_range('2024-01-01', '2024-03-31', freq='D')
stock = pd.DataFrame({'close': 50000 + np.cumsum(np.random.randn(len(dates)) * 500)}, index=dates)
```

**기대 결과**: 월별 ['Open', 'High', 'Low', 'Close'] 컬럼

In [ ]:
# 여기에 코드를 작성하세요


### Q5. 업샘플링 후 선형 보간 ⭐⭐⭐

**문제**: 월별 예산 데이터를 일별로 업샘플링한 후 선형 보간(interpolate)하세요.

```python
monthly_budget = pd.DataFrame({'budget': [1000, 1200, 1100]}, index=pd.date_range('2024-01', periods=3, freq='MS'))
```

**기대 결과**: 일별 예산 (선형 보간)

In [ ]:
# 여기에 코드를 작성하세요


### Q6. 이동 표준편차로 변동성 측정 ⭐⭐⭐

**문제**: 주가 데이터에서 20일 이동 표준편차를 계산하여 변동성을 측정하세요.

```python
dates = pd.date_range('2024-01-01', periods=60, freq='D')
stock = pd.DataFrame({'price': 100 + np.cumsum(np.random.randn(60) * 2)}, index=dates)
```

**기대 결과**: 'volatility' 컬럼 (20일 이동 표준편차)

In [ ]:
# 여기에 코드를 작성하세요


### Q7. 지수 가중 이동 평균 (EWM) 계산 ⭐⭐⭐⭐

**문제**: 주가 데이터에서 span=10인 지수 가중 이동 평균을 계산하세요.

```python
dates = pd.date_range('2024-01-01', periods=50, freq='D')
stock = pd.DataFrame({'price': 100 + np.cumsum(np.random.randn(50) * 2)}, index=dates)
```

**기대 결과**: 'EWM_10' 컬럼 추가

In [ ]:
# 여기에 코드를 작성하세요


### Q8. 시계열 분해 수행하기 ⭐⭐⭐⭐

**문제**: 다음 월별 매출 데이터를 Additive 모델로 분해하여 추세, 계절성, 잔차를 추출하세요.

```python
dates = pd.date_range('2022-01', periods=24, freq='MS')
sales = pd.DataFrame({
    'revenue': 100 + np.linspace(0, 50, 24) + 20 * np.sin(np.linspace(0, 4*np.pi, 24)) + np.random.randn(24) * 5
}, index=dates)
```

**힌트**: `from statsmodels.tsa.seasonal import seasonal_decompose`, period=12

**기대 결과**: trend, seasonal, residual 컴포넌트

In [ ]:
# 여기에 코드를 작성하세요


### Q9. 계절성 제거 (Deseasonalize) ⭐⭐⭐⭐

**문제**: Q8의 분해 결과를 사용하여 계절성을 제거한 데이터를 생성하세요.

**기대 결과**: 원본 데이터에서 계절성 컴포넌트를 뺀 값

In [ ]:
# 여기에 코드를 작성하세요


### Q10. Bollinger Bands로 이상치 탐지 ⭐⭐⭐⭐⭐

**문제**: 센서 온도 데이터에서 Bollinger Bands를 계산하고, 밴드를 벗어나는 이상치를 탐지하세요.

```python
np.random.seed(42)
dates = pd.date_range('2024-01-01', periods=100, freq='H')
sensor = pd.DataFrame({'temp': 25 + np.random.randn(100) * 2}, index=dates)
sensor.loc[sensor.index[20], 'temp'] = 35  # 이상치
sensor.loc[sensor.index[80], 'temp'] = 15  # 이상치
```

**요구사항**:
1. 20기간 이동평균 계산
2. Upper/Lower Band 계산 (평균 ± 2 × 표준편차)
3. 이상치 플래그 ('anomaly' 컬럼)
4. Plotly로 시각화 (온도, 밴드, 이상치 마커)

**기대 결과**: 이상치 탐지 완료 + 시각화

In [ ]:
# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 메서드 | 실무 활용 |
|------|------------|----------|
| **resample** | `.resample('W').sum()` | 일별 → 주별/월별 집계, 주기 변환 |
| **다운샘플링** | `.resample('M').agg({...})` | 고빈도 → 저빈도 (데이터 요약) |
| **업샘플링** | `.resample('D').ffill()` / `.interpolate()` | 저빈도 → 고빈도 (결측치 채우기) |
| **rolling** | `.rolling(7).mean()` | 이동평균, 노이즈 제거, 추세 파악 |
| **expanding** | `.expanding().sum()` | 누적 합계, 누적 평균, 연초부터 현재까지 |

### Part 2: 심화 핵심 요약

| 기법 | 문법 | 언제 쓰나? |
|------|------|----------|
| **EWM** | `.ewm(span=10).mean()` | 최근 데이터 가중치 높임 (변동성, 트렌드 추적) |
| **시계열 분해** | `seasonal_decompose(model='additive', period=12)` | 추세/계절성/잔차 분리, 예측 모델링 준비 |
| **계절성 제거** | `원본 - seasonal` | 계절 효과 제거 후 순수 성장 추세 파악 |
| **Bollinger Bands** | `MA ± 2*STD` | 이상치 탐지, 변동성 모니터링 |

### 💡 실무 팁

1. **주기 선택**: 데이터 특성에 맞는 주기 사용 (일별 → 주별은 'W', 월별은 'M')
2. **결측치 처리**: 업샘플링 후 `.ffill()` (계단형) vs `.interpolate()` (선형) 선택
3. **이동평균 윈도우**: 짧을수록 민감, 길수록 평활화 (주가: 5일, 20일, 60일선)
4. **EWM vs SMA**: 최근 변화 빠르게 반영하려면 EWM, 안정적이면 SMA
5. **시계열 분해**: 계절성 있는 데이터(월별, 분기별)는 반드시 분해 후 분석
6. **이상치 탐지**: Bollinger Bands의 2σ 기준은 약 95% 신뢰구간
7. **Plotly 시각화**: `hovermode='x unified'`로 여러 시계열 동시 비교
8. **DatetimeIndex 필수**: resample/rolling 사용 전 `.set_index('date')` 필수